# Data Filtering and Cleaning

## Data Filtering

In [9]:
import gzip
import csv
import logging

# Disable jedi autocompleter
%config Completer.use_jedi = False

In [3]:
# Unzimp the datafile
path = "../datasets/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
f = gzip.open(path, 'rt')

# Initiate a reader object
reader = csv.reader(f, delimiter='\t')

# read one line at a time
# First line = header
header = next(reader)

In [4]:
dataset = []

for line in reader:
    d = dict(zip(header, line))
    # convert string to int
    for field in ['helpful_votes', 'star_rating', 'total_votes']:
        d[field] = int(d[field])
    # convert string to boolean
    for field in ['verified_purchase', 'vine']:
        if d[field] == "Y":
            d[field] = True
        else:
            d[field] = False
    dataset.append(d)

In [5]:
len(dataset)

148310

In [6]:
dataset[0]

{'marketplace': 'US',
 'customer_id': '24371595',
 'review_id': 'R27ZP1F1CD0C3Y',
 'product_id': 'B004LLIL5A',
 'product_parent': '346014806',
 'product_title': 'Amazon eGift Card - Celebrate',
 'product_category': 'Gift Card',
 'star_rating': 5,
 'helpful_votes': 0,
 'total_votes': 0,
 'vine': False,
 'verified_purchase': True,
 'review_headline': 'Five Stars',
 'review_body': 'Great birthday gift for a young adult.',
 'review_date': '2015-08-31'}

## Fintering by Date

Let's filter reviews by data in order to remove very old reviews. We first need to convert htis to an integer

In [12]:
import traceback
for d in dataset:
    try:
        d['yearInt'] = int(d['review_date'][:4])
    except Exception:
        traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-12-319bf42f2c40>", line 4, in <module>
    d['yearInt'] = int(d['review_date'][:4])
KeyError: 'review_date'


It seems like some reviews don't contain a `review_date` field. So we'll first have to preprocess our dataset to extract only those entries containg a `review_date` field

In [13]:
dataset = [d for d in dataset if 'review_date' in d]

In [14]:
len(dataset)

148309

* It looks like it was just one review that didn't have a `review_date` field
* Now we can try again

In [15]:
for d in dataset:
    d['yearInt'] = int(d['review_date'][:4])

Finally let's filter out old review, that is those written before 2010

In [16]:
dataset = [d for d in dataset if d['yearInt'] > 2009]
len(dataset)

148095

Similarly, we might filter reviews based on their "helpfulness":  
* Keep reviews that haven't received many votes yet,
* Otherwise, delete any with less than 50% helpfulness

In [17]:
dataset = [d for d in dataset if d['total_votes'] < 3 or d['helpful_votes']/d['total_votes'] >= 0.5]

Next, let's filter our dataset to discard inactive users (in this case users who have written only a single review in this category)

In [18]:
from collections import defaultdict
nReviewsPerUser = defaultdict(int)

In [20]:
for d in dataset:
    nReviewsPerUser[d['customer_id']] += 1

Then we can filter to keep only users with 2 or more reviews:

In [22]:
dataset = [d for d in dataset if nReviewsPerUser[d['customer_id']] >= 2]

In [24]:
len(dataset)

11172

Finally, let's filter very short reviews, which may be uninformative

In [25]:
dataset = [d for d in dataset if len(d['review_body'].split()) >= 10]

In [26]:
len(dataset)

7033

## Processing Text and Strings

In this section we will...  

* Perform simple manipulations of string data in python
* Discover a few useful library function for strin processing

First let's read in a review from the Yelp dataset:


In [27]:
import json
import string

path = "../datasets/yelp_dataset/review.json"
f = open(path)

In [45]:
d = json.loads(f.readline())

In [46]:
next(f)

'{"review_id":"J4a2TuhDasjn2k3wWtHZnQ","user_id":"RNm_RWkcd02Li2mKPRe7Eg","business_id":"xGXzsc-hzam-VArK6eTvtw","stars":1.0,"useful":2,"funny":0,"cool":0,"text":"This place used to be a cool, chill place. Now its a bunch of neanderthal bouncers hopped up on steroids acting like the can do whatever they want. There are so many better places in davis square where they are glad you are visiting their business. Sad that the burren is now the worst place in davis.","date":"2018-01-21 04:41:03"}\n'

In [49]:
review = d['text']

In [50]:
review

"The food is always great here. The service from both the manager as well as the staff is super. Only draw back of this restaurant is it's super loud. If you can, snag a patio table!"

Let's covert strings to a list of words

In [51]:
reviewWords = review.split()

In [52]:
reviewWords

['The',
 'food',
 'is',
 'always',
 'great',
 'here.',
 'The',
 'service',
 'from',
 'both',
 'the',
 'manager',
 'as',
 'well',
 'as',
 'the',
 'staff',
 'is',
 'super.',
 'Only',
 'draw',
 'back',
 'of',
 'this',
 'restaurant',
 'is',
 "it's",
 'super',
 'loud.',
 'If',
 'you',
 'can,',
 'snag',
 'a',
 'patio',
 'table!']

In [53]:
' '.join(reviewWords)

"The food is always great here. The service from both the manager as well as the staff is super. Only draw back of this restaurant is it's super loud. If you can, snag a patio table!"